# Multi-Branch Interactions & Patterns

This notebook demonstrates how to orchestrate multiple AI agents working together, showcasing the power of lionagi's branch system for complex AI workflows.

**What you'll learn:**
- Creating separate AI agents with distinct personalities and roles
- Passing context between different branches for collaboration
- Implementing feedback loops for iterative improvement
- Persisting and reloading conversation state

In [1]:
from lionagi import Branch
from IPython.display import Markdown

In [2]:
sys_comedian = "As a comedian, you are sarcastically funny"

prompt1 = "short joke: a blue whale and a big shark meet at the bar and start dancing"
prompt2 = "continue the joke: and then they stopped"

In [3]:
# start a LLM session and have consecutive conversation with it
comedian = Branch(system=sys_comedian)

joke1 = await comedian.communicate(prompt1)
joke2 = await comedian.communicate(prompt2)

In [4]:
Markdown(joke1)

A blue whale and a big shark meet at the bar, see each other and start dancing. Guess they thought it was the perfect way to **"sink" the competition**.

In [5]:
Markdown(joke2)

And then they stopped, realizing the only thing bigger than their dance moves was the tab—guess even giants get sea-sick from paying the bill!

**Introducing Multi-Agent Collaboration:**

Now we'll create a second AI agent with a different role and personality. This demonstrates a common pattern in AI applications: having specialized agents that excel at different tasks.

**The Multi-Agent Pattern:**
- **Comedian**: Creative, humorous, focused on entertainment
- **Critic**: Analytical, logical, focused on evaluation and improvement

This separation of concerns allows each agent to specialize in what they do best, leading to higher quality results through collaboration.

In [6]:
# some messages for the critic
sys_critic = "you are a respected commentator, you are artistically logical"

prompt3 = "short comments, what do you think about the first joke?"
prompt4 = (
    "provide a concise artistic critique on both jokes, and rate from 1-10"
)

In [7]:
# create a session with the critic and generate some comments in a multi-turn conversation
critic = Branch(system=sys_critic)

# provide context for the critic, context needs to be string or string mappings/iterables
context = {
    "joke1": {"prompt": prompt1, "response": joke1},
    "joke2": {"prompt": prompt2, "response": joke2},
}

# you only need to provide same context once in a conversation
comment1 = await critic.communicate(prompt3, context=context)
comment2 = await critic.communicate(prompt4)

In [8]:
Markdown(comment1)

This joke cleverly fuses maritime grandeur with humor, creating a playful visual of giant sea creatures dancing. The punchline's pun on "sink" adds a clever twist, making it both amusing and imaginatively vivid. A delightful blend of whimsy and wit!

In [9]:
Markdown(comment2)

**Joke 1:**  
The imagery of a blue whale and a shark dancing at a bar is charming and surreal, effectively blending humor with visual grandeur. The pun on "sink the competition" adds a clever layer, though it relies somewhat on familiarity with idiomatic expressions.  
**Rating:** 7/10

**Joke 2:**  
The continuation humorously depicts the aftermath with a relatable twist about the bill, cleverly tying the sea theme to everyday humor. The punchline is playful and maintains the whimsical tone, though it slightly diminishes the visual impact of the first.  
**Rating:** 6.5/10

**Implementing Feedback Loops:**

One of the most powerful patterns in AI workflows is the feedback loop, where one agent's output becomes another agent's input. This creates an iterative improvement cycle.

**The Feedback Process:**
1. **Initial Creation**: Comedian generates content
2. **Critical Analysis**: Critic evaluates the content  
3. **Reflection & Improvement**: Comedian receives feedback and refines their work

This pattern mimics real-world creative processes and often leads to significantly improved results compared to single-shot generation.

In [10]:
# messages for comedian again
prompt5 = (
    "your jokes were evaluated by a critic, does it make sense to you? why?"
)
prompt6 = "basing on your reflection, write joke1 again"
prompt7 = "write joke2 again"

# let us provide critic's feedback to comedian
context2 = [i.response for i in critic.msgs.assistant_responses]

In [11]:
# run the instructions
reflect = await comedian.communicate(prompt5, context=context2)
joke11 = await comedian.communicate(prompt6)
joke22 = await comedian.communicate(prompt7)

In [12]:
Markdown(reflect)

Yes, it makes sense to me. The critic appreciates the vivid imagery and the clever pun in the original joke, noting that it blends humor with a playful visual of giant sea creatures dancing. The humor relies on whimsical absurdity and a pun, which are effective in comedy. The continuation maintains this whimsical tone while adding a relatable twist, keeping the humor flowing. Overall, both jokes follow a logical comedic progression—establishing a funny scene and then delivering a clever punchline. So, yes, I believe the critique’s assessments and ratings align well with the humor’s coherence and wit.

In [13]:
Markdown(joke11)

# Assistant Response

**Joke 1 Revised**:  
A blue whale and a shark walk into a bar, start dancing like nobody’s watching—probably because no one can see either of them over the appetizers. Guess they wanted to "sink" into the groove!

In [14]:
Markdown(joke22)

# Assistant Response

**Joke 2 Revised**:  
After a wild dance, they stopped — turns out even giants can't handle the tab when they’re swimming in bills. Guess they finally realized they can't "sea" the end of that check!

In [15]:
comedian.to_df()

,created_at,role,content,id,sender,recipient,metadata
0,1.753205e+09,system,"{'system_message': 'As a comedian, you are sar...",08e464d9-788b-404b-a1b5-53dece5937c2,system,f0f863f0-85c2-46d9-bd88-3f688868dcb3,{'lion_class': 'lionagi.protocols.messages.sys...
1,1.753205e+09,user,"{'context': [], 'instruction': 'short joke: a ...",c43d3633-c451-4442-883c-2298f342e0b6,user,f0f863f0-85c2-46d9-bd88-3f688868dcb3,{'lion_class': 'lionagi.protocols.messages.ins...
2,1.753205e+09,assistant,{'assistant_response': 'A blue whale and a big...,d30ce508-37aa-4760-994b-22c2333ef589,f0f863f0-85c2-46d9-bd88-3f688868dcb3,user,{'model_response': {'id': 'chatcmpl-BwAt805cxm...
3,1.753205e+09,user,"{'context': [], 'instruction': 'continue the j...",6e9a5903-a6ac-49c2-b154-b6f8de95f548,user,f0f863f0-85c2-46d9-bd88-3f688868dcb3,{'lion_class': 'lionagi.protocols.messages.ins...
4,1.753205e+09,assistant,"{'assistant_response': 'And then they stopped,...",9849dd79-8ba9-40df-8efd-e1e954ea0ef6,f0f863f0-85c2-46d9-bd88-3f688868dcb3,user,{'model_response': {'id': 'chatcmpl-BwAt90c3bi...
5,1.753205e+09,user,{'context': ['This joke cleverly fuses maritim...,5e8c4255-1648-407c-b10d-c27c8ed9fc9f,user,f0f863f0-85c2-46d9-bd88-3f688868dcb3,{'lion_class': 'lionagi.protocols.messages.ins...
6,1.753205e+09,assistant,"{'assistant_response': 'Yes, it makes sense to...",0c134280-fb58-469c-bf1d-1d63458e2b90,f0f863f0-85c2-46d9-bd88-3f688868dcb3,user,{'model_response': {'id': 'chatcmpl-BwAtCnvvLG...
7,1.753205e+09,user,"{'context': [], 'instruction': 'basing on your...",3db4143f-bfe8-489c-be32-f75edc245b8c,user,f0f863f0-85c2-46d9-bd88-3f688868dcb3,{'lion_class': 'lionagi.protocols.messages.ins...
8,1.753205e+09,assistant,{'assistant_response': '# Assistant Response ...,c4dab3ca-1a04-48fc-86cf-df8375fb5faa,f0f863f0-85c2-46d9-bd88-3f688868dcb3,user,{'model_response': {'id': 'chatcmpl-BwAtEOamxb...
9,1.753205e+09,user,"{'context': [], 'instruction': 'write joke2 ag...",e61d0f71-bb8a-4b7d-b53d-606949ef3b1f,user,f0f863f0-85c2-46d9-bd88-3f688868dcb3,{'lion_class': 'lionagi.protocols.messages.ins...


In [16]:
critic.to_df()

,created_at,role,content,id,sender,recipient,metadata
0,1.753205e+09,system,{'system_message': 'you are a respected commen...,3d660d1a-c5d7-4f6e-bc7f-4f0902acd611,system,b4908ee1-9aef-4246-a291-63f3c7790280,{'lion_class': 'lionagi.protocols.messages.sys...
1,1.753205e+09,user,{'context': [{'joke1': {'prompt': 'short joke:...,92d080dc-8b9c-465e-a691-29743d8f6f56,user,b4908ee1-9aef-4246-a291-63f3c7790280,{'lion_class': 'lionagi.protocols.messages.ins...
2,1.753205e+09,assistant,{'assistant_response': 'This joke cleverly fus...,6145ae26-add8-421a-a358-d44e32e3a722,b4908ee1-9aef-4246-a291-63f3c7790280,user,{'model_response': {'id': 'chatcmpl-BwAtAQlEE7...
3,1.753205e+09,user,"{'context': [], 'instruction': 'provide a conc...",93bbc794-b5c9-4f8d-8563-8509ad9c26e6,user,b4908ee1-9aef-4246-a291-63f3c7790280,{'lion_class': 'lionagi.protocols.messages.ins...
4,1.753205e+09,assistant,{'assistant_response': '**Joke 1:** The imag...,1721fdc3-6254-4e86-b8c2-2d570bd2d538,b4908ee1-9aef-4246-a291-63f3c7790280,user,{'model_response': {'id': 'chatcmpl-BwAtBCbIte...


**Branch Persistence: Saving & Loading State**

One of lionagi's powerful features is the ability to serialize and persist entire conversation branches. This enables:

**Production Use Cases:**
- **Session Recovery**: Resume conversations after system restarts
- **Conversation Archiving**: Store important dialogues for future reference  
- **A/B Testing**: Save different conversation branches for comparison
- **Audit Trails**: Maintain complete records of AI interactions

The serialization captures not just the messages, but the entire conversation state including metadata, model responses, and conversation flow.

In [17]:
import json

with open("data/002_comedian.json", "w") as f:
    json.dump(comedian.to_dict(), f, indent=4)

with open("data/002_critic.json", "w") as f:
    json.dump(critic.to_dict(), f, indent=4)

In [18]:
# To use them again

with open("data/002_comedian.json") as f:
    dict_ = json.load(f)
    comedian = Branch.from_dict(dict_)

comedian.to_df()

,created_at,role,content,id,sender,recipient,metadata
0,1.753205e+09,system,{'system_message': {'id': '08e464d9-788b-404b-...,85dd4852-5aed-4dc5-8e70-1fad07512101,system,f0f863f0-85c2-46d9-bd88-3f688868dcb3,{'lion_class': 'lionagi.protocols.messages.sys...
1,1.753205e+09,system,"{'system_message': 'As a comedian, you are sar...",08e464d9-788b-404b-a1b5-53dece5937c2,system,f0f863f0-85c2-46d9-bd88-3f688868dcb3,{'lion_class': 'lionagi.protocols.messages.sys...
2,1.753205e+09,user,"{'context': [], 'instruction': 'short joke: a ...",c43d3633-c451-4442-883c-2298f342e0b6,user,f0f863f0-85c2-46d9-bd88-3f688868dcb3,{'lion_class': 'lionagi.protocols.messages.ins...
3,1.753205e+09,assistant,{'assistant_response': 'A blue whale and a big...,d30ce508-37aa-4760-994b-22c2333ef589,f0f863f0-85c2-46d9-bd88-3f688868dcb3,user,{'model_response': {'id': 'chatcmpl-BwAt805cxm...
4,1.753205e+09,user,"{'context': [], 'instruction': 'continue the j...",6e9a5903-a6ac-49c2-b154-b6f8de95f548,user,f0f863f0-85c2-46d9-bd88-3f688868dcb3,{'lion_class': 'lionagi.protocols.messages.ins...
5,1.753205e+09,assistant,"{'assistant_response': 'And then they stopped,...",9849dd79-8ba9-40df-8efd-e1e954ea0ef6,f0f863f0-85c2-46d9-bd88-3f688868dcb3,user,{'model_response': {'id': 'chatcmpl-BwAt90c3bi...
6,1.753205e+09,user,{'context': ['This joke cleverly fuses maritim...,5e8c4255-1648-407c-b10d-c27c8ed9fc9f,user,f0f863f0-85c2-46d9-bd88-3f688868dcb3,{'lion_class': 'lionagi.protocols.messages.ins...
7,1.753205e+09,assistant,"{'assistant_response': 'Yes, it makes sense to...",0c134280-fb58-469c-bf1d-1d63458e2b90,f0f863f0-85c2-46d9-bd88-3f688868dcb3,user,{'model_response': {'id': 'chatcmpl-BwAtCnvvLG...
8,1.753205e+09,user,"{'context': [], 'instruction': 'basing on your...",3db4143f-bfe8-489c-be32-f75edc245b8c,user,f0f863f0-85c2-46d9-bd88-3f688868dcb3,{'lion_class': 'lionagi.protocols.messages.ins...
9,1.753205e+09,assistant,{'assistant_response': '# Assistant Response ...,c4dab3ca-1a04-48fc-86cf-df8375fb5faa,f0f863f0-85c2-46d9-bd88-3f688868dcb3,user,{'model_response': {'id': 'chatcmpl-BwAtEOamxb...
